In [1]:
import bs4 as bs
import re

import datetime
import pprint
import pandas as pd
import requests
import time
from selenium import webdriver
import urllib.request
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime

In [2]:
driver = webdriver.Chrome('C:\cd\\chromedriver.exe')


options = webdriver.ChromeOptions()

options.add_argument('--window-size=1920x1080')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')

In [3]:
final_df = pd.DataFrame(columns = ['USD/RUB_buy', 'USD/RUB_sell', 'EUR/RUB_buy', 'EUR/RUB_sell', 'channel', 'bank_name'])
error_log = pd.DataFrame(columns = ['bank_name', 'time', 'date'])

In [4]:
#тут будем загружать имеющийся файлик

In [5]:
#тут будем к нему присоединять данные и сохранять

### Investing

In [6]:
def investing(link, clasS):
    driver.get('https://ru.investing.com/currencies/{}'.format(link))
    page_source = driver.page_source
    soup = bs.BeautifulSoup(page_source, 'html.parser')
    value = soup.find('span', {'class' : clasS}).text
    value = [s.replace(',', '.') for s in value]
    string = ''.join(value)
    number = float(string)
    return number

In [ ]:
usd = investing('usd-rub', 'arial_26 inlineblock pid-2186-last')
eur = investing('eur-rub', 'arial_26 inlineblock pid-1691-last')

### Raiffeisenbank

In [ ]:
driver.get('https://www.raiffeisen.ru/currency_rates/?active_tab=')
page_source = driver.page_source
soup = bs.BeautifulSoup(page_source, 'html.parser')

channels = ['online', 'offices', 'card']

offices_idx = [10,12,17,19]
online_idx = [8,9,13,14]
card_idx = [4,5,7,8]

In [ ]:
def raiff_parser(channel, channel_idx):
    final_text = []

    results = soup.find("div", {'id': channel})
    text = results.findAll('div', {'class':'b-table__td'})

    all_text = []
    for i in text:
        all_text.append(i.text)
    all_text = [s.replace('\n', '') for s in all_text]
    all_text = [s.replace(' ', '') for s in all_text]
    
        
    for i in channel_idx:
        final_text.append(all_text[i])


    final_text.append(channel)
    temp_df = pd.DataFrame(final_text).T
    return temp_df

In [ ]:
try:
    online = raiff_parser('online', online_idx)
    offices = raiff_parser('offices', offices_idx)
    card = raiff_parser('card', card_idx)

    fx_raiff_df = pd.concat([online, offices], axis = 0)
    fx_raiff_df = pd.concat([fx_raiff_df, card], axis = 0)

    fx_raiff_df['bank_name'] = 'Raiffeisen'

    fx_raiff_df = fx_raiff_df.rename(columns={0: "USD/RUB_buy",
                                          1:'USD/RUB_sell', 2:'EUR/RUB_buy', 3:'EUR/RUB_sell',4:'channel'})

    final_df = pd.concat([final_df, fx_raiff_df], axis = 0)
except:
    error_log['bank_name']

### SBER

In [ ]:
def sber_parser(link, channel):
    driver.get("https://www.sberbank.ru/ru/quotes/currencies{}".format(link))
    time.sleep(5)
    page_source = driver.page_source
    soup = bs.BeautifulSoup(page_source, 'lxml')

    table = soup.find('table', {'class':'rates-current__table'})
    table_body = table.find('tbody')
    data = []

    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) 
        
    sber_tmp = []
    sber_tmp.append(data[2][1])
    sber_tmp.append(data[2][2])
    sber_tmp.append(data[1][1])
    sber_tmp.append(data[1][2])
    sber_tmp = [s.replace(',', '.') for s in sber_tmp]
    sber_tmp = [s[0 : 5] for s in sber_tmp]
    temp_df_sber = pd.DataFrame(sber_tmp).T
    temp_df_sber['channel'] = channel
    return temp_df_sber

In [ ]:
try:
    first = sber_parser('first', 'first')
    premier = sber_parser('premier', 'premier')

    fx_sber_df = pd.concat([first, premier], axis = 0)
    fx_sber_df['bank_name'] = 'Sber'


    fx_sber_df = fx_sber_df.rename(columns={0: "USD/RUB_buy", 1:'USD/RUB_sell', 2:'EUR/RUB_buy', 3:'EUR/RUB_sell'}) 

    final_df = pd.concat([final_df, fx_sber_df], axis = 0)
except:
    print('error')

### Alfa

In [ ]:
def alfa_parser(channel):
    driver.get('https://alfabank.ru/currency/')
    page_source = driver.page_source
    soup = bs.BeautifulSoup(page_source, 'lxml')

    table = soup.find('table', {'class':'table'})
    table_body = table.find('tbody')
    data = []

    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])
    
    alfa_tmp = []
    alfa_tmp.append(data[2][1])
    alfa_tmp.append(data[2][2])
    alfa_tmp.append(data[3][1])
    alfa_tmp.append(data[3][2])
    temp_df_alfa = pd.DataFrame(alfa_tmp).T
    temp_df_alfa['channel'] = channel
    
    return temp_df_alfa

In [ ]:
try:
    fx_alfa_df = alfa_parser('online')
    fx_alfa_df['bank_name'] = 'Alfa'

    fx_alfa_df = fx_alfa_df.rename(columns={0: "USD/RUB_buy", 1:'USD/RUB_sell', 2:'EUR/RUB_buy', 3:'EUR/RUB_sell'}) 
    final_df = pd.concat([final_df, fx_alfa_df], axis = 0)
except:
    print('error')

### VTB

In [ ]:
try:
    driver.get('https://www.vtb.ru/personal/platezhi-i-perevody/obmen-valjuty/')

    #Скролим немного вниз что бы кнопку можно было нажать,иначе перекрывает уведомление о куки
    driver.execute_script("window.scrollTo(0, 500);") 
    driver.implicitly_wait(10)

    #нажимаем кнопку
    driver.find_element_by_xpath("//body/main/div/section/div/div/div/div/div/div/div/section/div/div/ul/li[3]/a").click()
    time.sleep(4)

    page_source = driver.page_source
    soup = bs.BeautifulSoup(page_source, 'lxml')

    #ищем нужные данные
    table = soup.findAll('table', {'class' : 'exchange-rate-table_no-width'})[2]
    table_body = table.find('tbody')
    data = []

    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])

    vtb_tmp = []
    vtb_tmp.append(data[1][1])
    vtb_tmp.append(data[1][2])
    vtb_tmp.append(data[2][1])
    vtb_tmp.append(data[2][2])
    vtb_tmp = [s.replace(',', '.') for s in vtb_tmp]

    fx_vtb_df = pd.DataFrame(vtb_tmp).T
    fx_vtb_df['channel'] = 'online'

    fx_vtb_df['bank_name'] = 'VTB'


    fx_vtb_df = fx_vtb_df.rename(columns={0: "USD/RUB_buy", 1:'USD/RUB_sell', 2:'EUR/RUB_buy', 3:'EUR/RUB_sell'}) 

    final_df = pd.concat([final_df, fx_vtb_df], axis = 0)
except:
    print('error')

### Unicredit

In [ ]:
driver.get('https://www.unicreditbank.ru/ru/personal/all-currencies.html')
page_source = driver.page_source
soup = bs.BeautifulSoup(page_source, 'lxml')

try:
    table = soup.find('div', {'class' : 'table_wrap'})
    table_body = table.find('tbody')
    data = []

    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])
    
    uni_tmp = []
    uni_tmp.append(data[0][3])
    uni_tmp.append(data[0][4])
    uni_tmp.append(data[1][3])
    uni_tmp.append(data[1][4])


    fx_uni_df = pd.DataFrame(uni_tmp).T
    fx_uni_df['channel'] = 'online'

    fx_uni_df['bank_name'] = 'Unicredit'


    fx_uni_df = fx_uni_df.rename(columns={0: "USD/RUB_buy", 1:'USD/RUB_sell', 2:'EUR/RUB_buy', 3:'EUR/RUB_sell'}) 

    final_df = pd.concat([final_df, fx_uni_df], axis = 0)
except:
    print('error')

### Gazprombank

In [ ]:
try:
    driver.get('https://www.gazprombank.ru/personal/courses/')

    #Скролим немного вниз что бы кнопку можно было нажать,иначе перекрывает уведомление о куки
    driver.execute_script("window.scrollTo(0, 500);") 
    driver.implicitly_wait(10)

    #нажимаем кнопку
    driver.find_element_by_xpath('//body/div/div/div/div/div/div/div/div/div/div/div/button[4]').click()


    page_source = driver.page_source
    soup = bs.BeautifulSoup(page_source, 'lxml')

    table = soup.find('div', {'class' : 'nr-courses-table'})
    rows = table.findAll('div', {'class' : 'nr-courses-table__line'})

    data = []
    data.append(rows[1].findAll('span')[2].text)
    data.append(rows[1].findAll('span')[0].text)
    data.append(rows[2].findAll('span')[2].text)
    data.append(rows[2].findAll('span')[0].text)
    
    fx_gzb_df = pd.DataFrame(data).T
    fx_gzb_df['channel'] = 'online'

    fx_gzb_df['bank_name'] = 'Gazprom'

    fx_gzb_df = fx_gzb_df.rename(columns={0: "USD/RUB_buy", 1:'USD/RUB_sell', 2:'EUR/RUB_buy', 3:'EUR/RUB_sell'}) 


    final_df = pd.concat([final_df, fx_gzb_df], axis = 0)
except:
    print('error')

### Otkrytie

In [ ]:
try:
    #парсим льготный и стандартный курс
    driver.get('https://www.open.ru/')
    time.sleep(8)
    driver.implicitly_wait(10)
    driver.find_element_by_xpath("//div[@id='rc-tabs-1-tab-defaultRate']").click()
    time.sleep(3)
    page_source = driver.page_source
    soup = bs.BeautifulSoup(page_source, 'lxml')

    #льготный

    table = soup.findAll('table', {'class':'main-page-exchange__table main-page-exchange__table--online'})[0]
    table_body = table.find('tbody')
    data = []
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])

    open_tmp = []
    open_tmp.append(data[2][1])
    open_tmp.append(data[2][3])
    open_tmp.append(data[1][1])
    open_tmp.append(data[1][3])
    open_tmp = [s.replace(',', '.') for s in open_tmp]

    fx_open_s = pd.DataFrame(open_tmp).T
    fx_open_s['channel'] = 'online 1000+'
    
    #стандартный
    table = soup.findAll('table', {'class':'main-page-exchange__table main-page-exchange__table--online'})[1]
    table_body = table.find('tbody')
    data = []
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])

    open_tmp = []
    open_tmp.append(data[1][1])
    open_tmp.append(data[1][3])
    open_tmp.append(data[2][1])
    open_tmp.append(data[2][3])
    
    open_tmp = [s.replace(',', '.') for s in open_tmp]

    fx_open_st = pd.DataFrame(open_tmp).T
    fx_open_st['channel'] = 'online 0-1000'
    
    #объединяем
    fx_open_s = pd.concat([fx_open_s, fx_open_st], axis = 0)

    fx_open_s['bank_name'] = 'Otkrytie'


    fx_open_s = fx_open_s.rename(columns={0: "USD/RUB_buy", 1:'USD/RUB_sell', 2:'EUR/RUB_buy', 3:'EUR/RUB_sell'}) 

    final_df = pd.concat([final_df, fx_open_s], axis = 0)
except:
    print('error')

### Tinkoff

In [ ]:
def tinkoff_parser(link):
    driver.get('https://api.tinkoff.ru/v1/currency_rates?from={}'.format(link))


    page_source = driver.page_source
    soup = bs.BeautifulSoup(page_source, 'lxml')
    soup = soup.text

    result = re.findall("\d+\.\d+", soup)
    return result

In [ ]:
try:
    usd_tink = tinkoff_parser('USD&to=RUB')
    eur_tink = tinkoff_parser('EUR&to=RUB')
    buy_idx = [0, 4, 14]
    sell_idx = [1, 5, 15]

    def data_tinkoff(idx, currency):
        data = []
        for i in idx:
            data.append(currency[i])
        return data

    usd_buy = data_tinkoff(buy_idx, usd_tink)
    usd_sell = data_tinkoff(sell_idx, usd_tink)

    eur_buy = data_tinkoff(buy_idx, eur_tink)
    eur_sell = data_tinkoff(sell_idx, eur_tink)

    channel_lst = ['standart', 'pro', 'premium']

    tinkoff_dict = {'USD/RUB_buy' : usd_buy,
               'USD/RUB_sell' : usd_sell,
               'EUR/RUB_buy' : eur_buy,
               'EUR/RUB_sell' : eur_sell,
               'channel' : channel_lst,
               'bank_name' : 'Tinkoff'}

    tinkoff_df = pd.DataFrame(tinkoff_dict)

    final_df = pd.concat([final_df, tinkoff_df], axis = 0)
except:
    print('error')

### Соединяем вместе

In [ ]:
final_df[['USD/RUB_buy', 'USD/RUB_sell', 'EUR/RUB_buy', 'EUR/RUB_sell']] = final_df[['USD/RUB_buy',
                                                                     'USD/RUB_sell',
                                                                     'EUR/RUB_buy',
                                                                     'EUR/RUB_sell']].astype(float)

In [ ]:
final_df['USD_market'] = usd
final_df['EUR_market'] = eur

final_df['USD_mid'] = (final_df['USD/RUB_buy']+final_df['USD/RUB_sell'])/2
final_df['EUR_mid'] = (final_df['EUR/RUB_buy']+final_df['EUR/RUB_sell'])/2

final_df['USD_spread_%'] = (1 - final_df['USD/RUB_buy']/final_df['USD_mid']) - (
    1 - final_df['USD/RUB_sell']/final_df['USD_mid'])*100

final_df['EUR_spread_%'] = (1 - final_df['EUR/RUB_buy']/final_df['EUR_mid']) - (
    1 - final_df['EUR/RUB_sell']/final_df['EUR_mid'])*100

final_df['date'] = datetime.now().strftime('%Y-%m-%d') 
final_df['time'] = datetime.now().strftime('%H:%M:%S')

In [ ]:
competitors =pd.read_excel(r'C:\parser\competitors.xlsx')
competitors = pd.concat([competitors, final_df], axis = 0)
competitors = competitors.reset_index(inplace = False, drop = True)
competitors.to_excel(r'C:\parser\competitors.xlsx', index=False)

In [ ]:
competitors[competitors['bank_name'] == 'Sber']